In [ ]:
# Cell 1: Setup & Installation (Refined)
import subprocess
import sys

def install_packages():
    """Checks if required packages are installed and installs them if missing."""
    packages = ['google-generativeai', 'pandas', 'python-dotenv', 'openpyxl']
    print("Checking required packages...")
    for package in packages:
        try:
            # A quick way to check if a package is installed is to try importing it
            __import__(package.replace('-', '_'))
            print(f"✅ {package} is already installed.")
        except ImportError:
            print(f"📦 Package '{package}' not found. Installing...")
            # Use subprocess to install the package quietly
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
            print(f"✅ {package} installed successfully.")

install_packages()

In [ ]:
# Cell 2: Configuration and Class Definitions (New & Refined)
import os
import json
import pandas as pd
import google.generativeai as genai
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path
from typing import Optional, Dict, Any
import io

# --- 1. Centralized Configuration ---
class Config:
    """Holds all configuration settings for the co-pilot."""

    def __init__(self):
        load_dotenv()
        self.api_key: Optional[str] = os.getenv("GOOGLE_API_KEY")
        self.model_name: str = "gemini-2.5-flash"
        self.data_file_path: Path = Path("CONSOLIDATED_OUTPUT_DATA.csv")
        self.db_summary_path: Path = Path("db_summary.json")
        self.kpi_mapping_path: Path = Path("context_kpi_mapping.json")
        self.queries_file_path: Path = Path("User Queries.xlsx")
        self.output_dir: Path = Path("copilot_runs")
        self.output_dir.mkdir(exist_ok=True)  # Ensure the main output directory exists
        self.max_retries: int = 2 # <<<< NEW: Maximum number of retries on code execution failure

    def validate(self):
        """Validates the configuration, especially the API key."""
        if not self.api_key:
            raise ValueError(
                '❌ API Key not found! Please create a .env file with GOOGLE_API_KEY="your_key".'
            )
        genai.configure(api_key=self.api_key)
        print("✅ API Key configured successfully!")


# --- 2. Data Handling Class ---
class DataLoader:
    """Handles loading all necessary data and context files."""

    def __init__(self, config: Config):
        self.config = config
        self.df: Optional[pd.DataFrame] = None
        self.db_summary: Optional[Dict] = None
        self.kpi_mapping: Optional[Dict] = None

    def load_all(self) -> bool:
        """Loads all data sources and returns True if successful."""
        try:
            print(f"⏳ Loading main data from '{self.config.data_file_path}'...")
            self.df = pd.read_csv(self.config.data_file_path)
            print(
                f"✅ Successfully loaded main data ({self.df.shape[0]} rows, {self.df.shape[1]} columns)."
            )

            print(f"⏳ Loading DB summary from '{self.config.db_summary_path}'...")
            with open(self.config.db_summary_path, "r") as f:
                self.db_summary = json.load(f)
            print("✅ Successfully loaded DB summary.")

            print(f"⏳ Loading KPI mapping from '{self.config.kpi_mapping_path}'...")
            with open(self.config.kpi_mapping_path, "r") as f:
                self.kpi_mapping = json.load(f)
            print("✅ Successfully loaded KPI mapping.")
            return True

        except FileNotFoundError as e:
            print(f"❌ ERROR: A required file was not found: {e.filename}")
            return False
        except Exception as e:
            print(f"❌ An error occurred during data loading: {e}")
            return False


# --- 3. Gemini Agent Class ---
class InsightsAgent:
    """Encapsulates all interactions with the Gemini API."""

    def __init__(self, config: Config):
        self.model = genai.GenerativeModel(config.model_name)
        self.config = config

    def generate_analysis_code(
        self, user_query: str, db_summary: str, kpi_mapping: str
    ) -> str:
        """Generates Python code using a refined, detailed prompt."""
        prompt = f"""
            You are a **world-class market research data analyst and Python developer**.  
            Your task is to generate a **fully self-contained Python script** that answers the user's analytical question with **high accuracy** using **pandas** and **numpy**.  

            You must strictly follow the **provided data context** and the **analytical playbook**.

            ---

            ### 1. User Question
            {user_query}

            ---

            ### 2. Data Source
            - The data is in a CSV file located at: {self.config.data_file_path.resolve()}
            - You must load the data **directly from this absolute path** (do not modify it).

            ---

            ### 3. Database Summary (`db_summary.json`)
            This JSON describes the table columns. Use it to understand dimensions and measures.  
            - Demographics are in the `"Datacut"` column.  
            - Time periods are in the `"Time_Period"` column.  
            - Numeric values are in the `"value"` column.  

            ```json
            {db_summary}
            ```
            ---

            ### 4. Context–KPI Mapping
            This JSON defines how to map Context and KPI/Brand correctly.  
            - First, filter by the relevant "Context" based on the user's question.  
            - Then, filter by "KPI" or "Brand".  
            
            ```json
            {kpi_mapping}
            ```

            ---

            ### 5. Analytical Playbook (STRICT RULES)

            **Time Handling** - `Time_Period` is formatted as `"H1'25"` (first half 2025).  
            - Always sort chronologically (not alphabetically).  

            **Demographics Logic** - If the user does not specify a demographic, default to `"Total"`.  

            **All Brands/KPIs**
            - If the user does not specify a brand or KPI, then instead of not applying any filter on brands/kpis, you must pass a filter to include **all** brands/KPIs present in the (`db_summary.json`).

            **Exclude Non-Analytical Metrics** - ALWAYS EXCLUDE "Unweighted Base", "Sample Size", and "Base" from calculations, averages, rankings, and outputs, unless explicitly requested by the user to include it.

            **Change Over Time** To calculate change:  
            a. Filter data.  
            b. Create a pivot with `Time_Period` as columns.  
            c. Subtract the older period’s value from the newer period’s value.  

            **Ranking Logic** For "Top/Bottom N" requests, use:  
            ```python
            df.sort_values(by="value", ascending=False).head(N)
            ```
            
            **Detail of Final Dataframe**
            - The final dataframe must explanatory. So instead of just the final number, include relevant summarized breakdowns used to arrive at the final answer.
            - For example:
                - If you are asked to find the top 5 brands, include the brand names and their values in the final output.
                - If you are asked for change over time, include both time periods and the calculated change.

            **Output Column** - Always use `"value"` column for calculations (these are given as absolute numbers, but represent percentage share except Base and Index Metrics).  

            ---

            ### 6. Final Script Requirements
            - Must be a complete, runnable Python script.  
            - Must use **pandas** and **numpy** only (no external libraries).  
            - Must produce a single pandas DataFrame named `result_df`.  
            - The final output must be a SINGLE pandas DataFrame printed to the console print(result_df.to_string())
            - Ensure no truncation, partial tables, or additional debug printouts.  

            ---

            ### OUTPUT FORMAT
            Your response should contain **only the Python script**.  
            Do not include explanations or extra commentary.
            """
        response = self.model.generate_content(prompt)
        code = response.text.strip()
        if "python" in code:
            code = code.split("python")[1].split("```")[0]
        return code.strip()

    # <<<< NEW METHOD: To handle code regeneration on error >>>>
    def regenerate_code_on_error(self, failed_code: str, error_message: str, user_query: str) -> str:
        """Takes failed code and an error message, then asks the model to fix it."""
        prompt = f"""
            You are an expert Python debugging assistant.
            The following Python script, which was written to answer the question '{user_query}', failed during execution.

            ### FAILED SCRIPT
            ```python
            {failed_code}
            ```

            ### ERROR MESSAGE
            ```
            {error_message}
            ```

            ### INSTRUCTIONS
            1.  **Analyze** the error message and the original code.
            2.  **Correct the script** to resolve the error while still fulfilling the original user query.
            3.  **Adhere strictly** to all the original script requirements (use pandas/numpy, produce `result_df`, print with `to_string()`).
            4.  Your output must be **only the complete, corrected, runnable Python script**. Do not provide explanations, apologies, or any text other than the code itself.
            """
        print("🤖 Asking the agent to fix the code based on the error...")
        response = self.model.generate_content(prompt)
        code = response.text.strip()
        if "python" in code:
            code = code.split("python")[1].split("```")[0]
        return code.strip()


    def generate_insight_summary(self, data_table_string: str, user_query: str) -> str:
        """Generates a direct answer to the user’s question with supporting data evidence."""
        prompt = f"""
        You are a senior market research analyst. Your task is to provide a clear, fact-based answer to the user’s question.

        ### Original User Question:
        "{user_query}"

        ### Data Analysis Results:
        {data_table_string}

        ### Instructions:
        - **Values from Analysis:** Use only the data provided in the analysis results above. Do not assume or invent any data.
        - **Type of Values:** The values represent percentage shares or Index scores and not aboslute counts or numbers, except for Bases. So if a number is 25, it means 25% share or an Index of 25 not 25 counts.
        - **Direct Answer:** Start with a precise response to the user’s question, framed as a clear statement.  
        - **Supporting Evidence (1-2 bullets):** Use the most relevant numbers, comparisons, or trends from the data to prove the answer.  
        - **Clarity:** Be concise, avoid fluff, and focus only on insights supported by the data provided.  
        - **Format:** Use Markdown for readability (headings, bold, bullet points where relevant).  
        """
        response = self.model.generate_content(prompt)
        return response.text.strip()


print("✅ All classes (Config, DataLoader, InsightsAgent) are defined and ready.")

In [ ]:
# Cell 3: Initialization
try:
    config = Config()
    config.validate()

    data_loader = DataLoader(config)
    data_loaded_successfully = data_loader.load_all()

    agent = InsightsAgent(config)
    print("\n✅ Co-pilot initialized successfully!")
except (ValueError, FileNotFoundError) as e:
    print(f"\n❌ Initialization failed: {e}")
    data_loaded_successfully = False

In [ ]:
# Cell 4: Main Orchestration (Refined with Classes and Retry Logic)

# --- USER CONTROL PANEL ---
START_ID = 1
END_ID = 16
# --------------------------

def run_batch_processing(start_id: int, end_id: int):
    """Main function to run the batch processing of user queries."""
    if not data_loaded_successfully:
        print("\n❌ Cannot run batch processing because data failed to load.")
        return

    try:
        queries_df = pd.read_excel(config.queries_file_path, sheet_name='Sheet 1')
        selected_queries = queries_df[(queries_df['ID'] >= start_id) & (queries_df['ID'] <= end_id)]
        print(f"✅ Loaded Excel file. Will process {len(selected_queries)} questions from ID {start_id} to {end_id}.")
    except FileNotFoundError:
        print(f"❌ ERROR: '{config.queries_file_path}' not found. Please ensure the file exists.")
        return

    batch_results_list = []
    
    # Prepare context strings once
    db_summary_str = json.dumps(data_loader.db_summary, indent=2)
    kpi_mapping_str = json.dumps(data_loader.kpi_mapping, indent=2)

    for _, row in selected_queries.iterrows():
        query_id, user_query = row['ID'], row['Question']
        print("\n" + "="*80 + f"\n▶️ Processing Query ID: {query_id} | Query: '{user_query}'\n" + "="*80)

        run_folder_name = f"run_ID_{query_id}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
        output_dir = config.output_dir / run_folder_name
        output_dir.mkdir()
        print(f"✅ Outputs will be saved in: '{output_dir}'")

        data_output, final_summary = "Error during processing.", "Not generated."
        generated_code = ""
        last_error_message = ""
        code_executed_successfully = False

        # <<<< NEW: Retry loop >>>>
        for attempt in range(config.max_retries + 1):
            print(f"\n--- Attempt {attempt + 1} of {config.max_retries + 1} ---")
            
            try:
                # Step 1: Generate or Regenerate Code
                if attempt == 0:
                    print("Step 1: Calling Insights Agent to generate initial code...")
                    generated_code = agent.generate_analysis_code(user_query, db_summary_str, kpi_mapping_str)
                else:
                    print("Step 1: Calling Insights Agent to regenerate code based on previous error...")
                    generated_code = agent.regenerate_code_on_error(generated_code, last_error_message, user_query)
                
                script_path = output_dir / f'generated_script_attempt_{attempt+1}.py'
                script_path.write_text(generated_code)
                print(f"✅ Code generated for attempt {attempt+1}.")

                # Step 2: Execute Code
                print("Step 2: Executing generated script...")
                result = subprocess.run([sys.executable, str(script_path)], capture_output=True, text=True, check=True, timeout=60)
                data_output = result.stdout
                (output_dir / 'data_output.txt').write_text(data_output)
                print("✅ Script executed successfully.")
                code_executed_successfully = True
                break # <<<< Exit the retry loop on success

            except subprocess.TimeoutExpired:
                last_error_message = "SCRIPT EXECUTION FAILED: Timeout after 60 seconds."
                print(f"❌ {last_error_message}")
            except subprocess.CalledProcessError as e:
                last_error_message = f"SCRIPT EXECUTION FAILED:\n\n{e.stderr}"
                print(f"❌ {last_error_message}")
            except Exception as e:
                last_error_message = f"An unexpected error occurred during processing: {e}"
                print(f"❌ {last_error_message}")

        # After the loop, check for success
        if code_executed_successfully:
            # Step 3: Generate Insights
            if data_output and data_output.strip():
                print("\nStep 3: Generating final summary...")
                final_summary = agent.generate_insight_summary(data_output, user_query)
                (output_dir / 'final_summary.md').write_text(final_summary)
                print("✅ Summary generated successfully.")
            else:
                final_summary = "No insights generated (empty data output)."
        else:
            # If all retries failed
            print(f"\n❌ All {config.max_retries + 1} attempts failed for Query ID {query_id}.")
            data_output = last_error_message # Store the final error
            final_summary = "Failed to generate insights after multiple attempts."

        batch_results_list.append({'ID': query_id, 'Question': user_query, 'Data Output': data_output, 'Insight Summary': final_summary})

    # After loop, create the consolidated report
    if batch_results_list:
        print("\n" + "="*80 + "\n✅ Batch processing complete. Creating consolidated summary report...")
        summary_df = pd.DataFrame(batch_results_list)
        summary_path = config.output_dir / f"Batch_Run_Summary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.xlsx"
        summary_df.to_excel(summary_path, index=False)
        print(f"🎉🎉🎉 Successfully created summary report: '{summary_path}' 🎉🎉🎉")

# --- RUN THE BATCH PROCESS ---
run_batch_processing(START_ID, END_ID)